# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,basco,20.4487,121.9702,23.37,96,99,3.43,PH,1738355648
1,1,albany,42.6001,-73.9662,1.00,77,100,1.14,US,1738355649
2,2,thompson,55.7435,-97.8558,-21.91,64,20,5.14,CA,1738355650
3,3,grange,-34.9000,138.4833,21.56,65,20,2.06,AU,1738355651
4,4,bethel,41.3712,-73.4140,4.69,90,100,0.45,US,1738355669


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles= "OSM",
    frame_height=400, 
    frame_width=600, 
    size='Humidity', 
    alpha=0.4, 
    color='City'
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
temp = city_data_df["Max Temp"] <= 24
humidity = city_data_df["Humidity"] < 65
wind = city_data_df["Wind Speed"] < 2
combined_criteria = temp & humidity & wind
ideal_city = city_data_df[combined_criteria]

# Drop any rows with null values
ideal_city_clean = ideal_city.dropna()

# Display sample data
ideal_city_clean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,papatowai,-46.5619,169.4708,16.95,60,100,1.64,NZ,1738355702
45,45,sitka,57.0531,-135.3300,-2.06,54,20,1.54,US,1738355741
92,92,gandarbal,34.2333,74.7833,4.88,52,99,0.94,IN,1738355795
112,112,hotan,37.0997,79.9269,-1.92,40,76,1.19,CN,1738355818
222,222,chonchi,-42.6128,-73.8092,17.64,46,5,0.80,CL,1738355946


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_clean.loc[:,["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,papatowai,NZ,-46.5619,169.4708,60,
45,sitka,US,57.0531,-135.3300,54,
92,gandarbal,IN,34.2333,74.7833,52,
112,hotan,CN,37.0997,79.9269,40,
222,chonchi,CL,-42.6128,-73.8092,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 10,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
   # get latitude, longitude from the DataFrame
   lat = row["Lat"]
   lon = row["Lng"]

   # Add the current city's latitude and longitude to the params dictionary
   params["filter"] = f"circle:{lon},{lat},{radius}"
   params["bias"] = f"proximity:{lon},{lat}"

   # Set base URL
   base_url = "https://api.geoapify.com/v2/places?"

   # Make and API request using the params dictionary
   name_address = requests.get(base_url, params=params)

   # Convert the API response to JSON format
   name_address = name_address.json()

   # Grab the first hotel from the results and store the name in the hotel_df DataFrame
   try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
   except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

   # Log the search results
   print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
papatowai - nearest hotel: No hotel found
sitka - nearest hotel: Aspen Suites Hotel
gandarbal - nearest hotel: No hotel found
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
chonchi - nearest hotel: Hotel Huildín
turpan - nearest hotel: Hua Sheng Hotel
tartouss - nearest hotel: فندق انترادوس
moab - nearest hotel: Kokopelli Inn
tura - nearest hotel: Hotel Polo Orchid
fort portal - nearest hotel: Ridge Hotel
sinop - nearest hotel: No hotel found
mhamid - nearest hotel: Hotel Kasbah Azalay
abadeh - nearest hotel: هتل لاله
kayenta - nearest hotel: Wetherill Inn
lovington - nearest hotel: Comfort Inn & Suites
zahirabad - nearest hotel: Oyo
yangambi - nearest hotel: No hotel found
butembo - nearest hotel: Canada House


,City,Country,Lat,Lng,Humidity,Hotel Name
12,papatowai,NZ,-46.5619,169.4708,60,No hotel found
45,sitka,US,57.0531,-135.3300,54,Aspen Suites Hotel
92,gandarbal,IN,34.2333,74.7833,52,No hotel found
112,hotan,CN,37.0997,79.9269,40,Hua Yi International Hotel (accepts foreigners)
222,chonchi,CL,-42.6128,-73.8092,46,Hotel Huildín
257,turpan,CN,42.9333,89.1667,42,Hua Sheng Hotel
294,tartouss,SY,34.8890,35.8866,64,فندق انترادوس
340,moab,US,38.5733,-109.5498,30,Kokopelli Inn
347,tura,IN,25.5198,90.2201,57,Hotel Polo Orchid
384,fort portal,UG,0.6617,30.2748,37,Ridge Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
hotel_data = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles= "OSM",
    frame_height=400, 
    frame_width=600, 
    size='Humidity', 
    alpha=0.4, 
    color='City',
    hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map
hotel_data

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)